In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
# Read NASA HTML
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Create a Beautiful Soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
#Pull first headline available
news_title=soup.find_all("div", "content_title")[0].text

#Pull first description available
news_p=soup.find_all("div", "article_teaser_body")[0].text


In [4]:
#Visit JPL Site
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Create a Beautiful Soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#Find image
result=soup.find("div",class_="img")
path=result.img['src']

#Store Featured Image
featured_image_url=str('https://www.jpl.nasa.gov')+path

In [6]:
#Visit Twitter
url='https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
#Pull Latest Tweet
result=soup.find("p",class_="tweet-text")
temp=result.text

#Remove Image Link
mars_weather=temp[:temp.index('hPapic')-1]

In [8]:
#Pull Table from Space Facts
url='https://space-facts.com/mars/'
tables = pd.read_html(url)
df=tables[0]

#Format as html
html_table = df.to_html()

In [14]:
#Visit USGS
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
#Find List of Hemispheres
hemispheres=soup.find_all("div",class_="item")


#Create Empty Dictionary
hemisphere_image_urls=[]
hemisphere_line={}


#Loop Through Hemispheres
for i in hemispheres:
    
    #Store Title
    title=i.find('h3').text
    hemisphere_line['title']=title
    
    #Visit Sub Url for Image
    ending=i.find('a')['href']
    start='https://astrogeology.usgs.gov'
    sub_url=start+ending
    response = requests.get(sub_url)
    sub_soup = BeautifulSoup(response.text, 'html.parser')
    
    #Pull Full Image
    img_url=sub_soup.find(href=re.compile("\.tif$"))['href']
    hemisphere_line['img_url']=img_url
    
    #Append Hemisphere to List
    hemisphere_image_urls.append(dict(hemisphere_line))
    
    
    

done
done
done
done


In [23]:
scrape_dict={"Nasa News":[news_title,news_p],
            "JPL Featured Image":featured_image_url,
            "Mars Weather":mars_weather,
            "Mars HTML Table":html_table,
             "Hemisphere Images":hemisphere_image_urls,
            }

In [24]:
scrape_dict

{'Nasa News': ['NASA Social Media and Websites Win Webby Awards ',
  'NASA\'s social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."'],
 'JPL Featured Image': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23177-640x350.jpg',
 'Mars Weather': 'InSight sol 147 (2019-04-26) low -98.4ºC (-145.2ºF) high -23.5ºC (-10.3ºF)\nwinds from the W at 3.7 m/s (8.3 mph) gusting to 11.7 m/s (26.2 mph)\npressure at 7.40',
 'Mars HTML Table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^2